In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import gc

In [3]:
#data = pd.read_table('../../datasets/netflix-prize-data/combined_data_1.txt', 
#                    sep=',',
#                   header=0,
#                   names = [''])

In [10]:
#data.head()

,1:
0,"1488844,3,2005-09-06"
1,"822109,5,2005-05-13"
2,"885013,4,2005-10-19"
3,"30878,4,2005-12-26"
4,"823519,3,2004-05-03"


In [89]:
#把数据进行处理转换成需要格式，并写入文本
def data_processing(filepath_read, filepath_out):
    
    f_w = open(filepath_out, 'w+')
    with open(filepath_read) as f:
        moviedid = ''
        for line in f:
            s_num = re.match('([1-9][0-9]*:)', line) #匹配数字
            if s_num:
                movieid = s_num.group().split(':')[0]
            else:
                user, rating, timestamp = line.strip().split(',')
                f_w.write(','.join([user, movieid, rating, timestamp]) + '\n')
    f_w.close()    

In [90]:
filepath_read = '../../datasets/netflix-prize-data/combined_data_1.txt'
filepath_out = '../../datasets/netflix-prize-data/combined_data_1_processing.txt'
data_processing(filepath_read, filepath_out)

In [2]:
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',')
data = Dataset.load_from_file('../../datasets/netflix-prize-data/combined_data_1_processing.txt', reader=reader)
train_set = data.build_full_trainset()

In [3]:
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}

In [4]:
algo = BaselineOnly(bsl_options=bsl_options)

In [ ]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...


In [ ]:
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)